In [11]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import train_test_split, cross_validate

In [20]:
data = pd.read_csv('data/merged_data.csv')
data.head(3)

,Unnamed: 0,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age,raw_title,raw_year,categories,description
0,8,2313,0553278223,7,The Martian Chronicles,RAY BRADBURY,1984,Spectra,http://images.amazon.com/images/P/0553278223.0...,http://images.amazon.com/images/P/0553278223.0...,http://images.amazon.com/images/P/0553278223.0...,"cincinnati, ohio, usa",23.0,themartianchronicles,1970.0,['Fiction'],The tranquility of Mars is disrupted by humans...
1,11,2313,0295955252,8,No-No Boy,John Okada,1978,University of Washington Press,http://images.amazon.com/images/P/0295955252.0...,http://images.amazon.com/images/P/0295955252.0...,http://images.amazon.com/images/P/0295955252.0...,"cincinnati, ohio, usa",23.0,nonoboy,1970.0,['Japanese'],NaN
2,24,2313,0553213601,0,Cyrano De Bergerac (Bantam Classics),EDMOND ROSTAND,1950,Bantam,http://images.amazon.com/images/P/0553213601.0...,http://images.amazon.com/images/P/0553213601.0...,http://images.amazon.com/images/P/0553213601.0...,"cincinnati, ohio, usa",23.0,cyranodebergeracbantamclassics,1970.0,['Drama'],Cyrano is a wonderful poet who feels ugly due ...


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192928 entries, 0 to 192927
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           192928 non-null  int64  
 1   User-ID              192928 non-null  int64  
 2   ISBN                 192928 non-null  object 
 3   Book-Rating          192928 non-null  int64  
 4   Book-Title           192928 non-null  object 
 5   Book-Author          192928 non-null  object 
 6   Year-Of-Publication  192928 non-null  int64  
 7   Publisher            192927 non-null  object 
 8   Image-URL-S          192928 non-null  object 
 9   Image-URL-M          192928 non-null  object 
 10  Image-URL-L          192928 non-null  object 
 11  Location             192928 non-null  object 
 12  Age                  139369 non-null  float64
 13  raw_title            192928 non-null  object 
 14  raw_year             192793 non-null  float64
 15  categories       

In [22]:
data[['User-ID', 'ISBN', 'Book-Rating']].isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [26]:
data['Book-Rating'].value_counts(normalize=True).mul(100).sort_index()

Book-Rating
0     62.709923
1      0.117142
2      0.218216
3      0.460794
4      0.726178
5      4.071467
6      2.894344
7      6.352111
8      9.176999
9      6.199722
10     7.073105
Name: proportion, dtype: float64

In [24]:
filtered_data = data[data['Book-Rating'] != 0]

In [30]:
# 1. Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
surpise_data = Dataset.load_from_df(filtered_data[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = train_test_split(surpise_data, test_size=0.2)

In [31]:
svd = SVD()
svd.fit(trainset)
predictions_svd = svd.test(testset)

In [32]:
cv_results = cross_validate(svd, surpise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6420  1.6446  1.6552  1.6341  1.6550  1.6462  0.0081  
MAE (testset)     1.2716  1.2723  1.2805  1.2739  1.2833  1.2763  0.0047  
Fit time          1.29    1.08    1.20    1.16    1.29    1.21    0.08    
Test time         0.09    0.08    0.09    0.09    0.10    0.09    0.01    
